# Intro

# Tokenization Steps

### Normalization

### Pretokenization

### Tokenization

### Postprocessing

## Tokenization Reflection

# Using Hugging Face Tokenizers

### Loading Tokenizer

### Encoding: Text to Tokens

### Decoding: Tokens to Text

### Considerations


- Tokenizers differ
- Long sequences
- Adjusting to your use case (fine-tuning)


## Hugging Face Tokenizers Reflection

# Overall Reflection